In [1]:
import pandas as pd
import numpy as np

dataparse = pd.read_csv('/Users/vanhome/Downloads/EEG.machinelearing_data_BRMH.csv')

In [2]:
filtered = dataparse[dataparse['main.disorder'].isin(['Schizophrenia', 'Healthy control'])]

alpha_psd_cols = [col for col in filtered.columns if 'COH' in col]

X = filtered[alpha_psd_cols].values
y = filtered['main.disorder'].apply(lambda x: 1 if x == 'Schizophrenia' else 0).values

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

X[np.isinf(X)] = np.nan
X = X[:, ~np.isnan(X).all(axis=0)]  # remove all-NaN cols
X = SimpleImputer(strategy="mean").fit_transform(X)
X = VarianceThreshold(threshold=0).fit_transform(X)
X = StandardScaler().fit_transform(X)

In [4]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
clf = LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000)

scores = cross_val_score(clf, X, y, cv=cv, scoring='roc_auc')
print("Mean AUC:", scores.mean())
print("Std AUC:", scores.std())

Mean AUC: 0.7268306636155607
Std AUC: 0.02551798092728887


In [5]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X)

In [6]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
clf = LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000)

scores = cross_val_score(clf, X_pca, y, cv=cv, scoring='roc_auc')
print("Mean AUC:", scores.mean())
print("Std AUC:", scores.std())

Mean AUC: 0.70593058733791
Std AUC: 0.03854354294258143
